In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether_v2 as birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.iloc[i].name
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-1.2}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n

In [3]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_store,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['recall','precision','pf']
    for _ in range(1):
        bell_df = {}
        score_df = pd.read_csv(data_source1 + '/bellwether_cdom_2.csv')
        score_df = score_df.rename(columns = {'Unnamed: 0':'id'})
        _cluster_bellwethers = score_df.bellwether.values.tolist()
        bell = birch_bellwether.bellwether(path,score_df)
        final_score = bell.bellwether(_cluster_bellwethers,_cluster_bellwethers)
        _results = {}
        for goal in goals:    
            for s_project in final_score.keys():
                if s_project not in _results.keys():
                    _results[s_project] = {}
                    _temp = []
                for d_projects in final_score[s_project].keys():
                    if goal == 'g':
                        _goal = 'g-score'
                    else:
                        _goal = goal
                    _score = np.median(final_score[s_project][d_projects][_goal])
                    _temp.append(np.median(final_score[s_project][d_projects][_goal]))
                if goal not in _results[s_project].keys():
                    _results[s_project][goal] = []
                _results[s_project][goal] = np.median(_temp)
        _df = pd.DataFrame.from_dict(_results, orient = 'index')
        dom_score = []
        for row_id in range(_df.shape[0]):
            project_name = _df.iloc[row_id].name
            row = _df.iloc[row_id].tolist()
            wins = dominate(_df,row,project_name,goals)
            dom_score.append(wins)
        _df['wins'] = dom_score
        print(_df)
        c_dom = _df.wins.values.tolist()
        best_project = _df.index[c_dom.index(max(c_dom))]
        best_project_perf = _df.loc[best_project].values.tolist()
        best_project_perf.append(best_project)
        bell_df[key] = best_project_perf
    perf_df = pd.DataFrame.from_dict(bell_df, orient = 'index', columns = ['recall','precision','pf','cdom','bellwether'])    
    print(data_source1 + '/bellwether_cdom_0.csv')
    perf_df.to_csv(data_source1 + '/bellwether_cdom_0_2.csv')
            
            
            

In [4]:
for i in range(20):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf_v2/2/fold_' + str(i)
    data_store = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf_v2/0/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,data_store,clusters,path,i)
    

0
rcp-company-uibindings.csv
mbse.csv
jfreereport.csv
qse.csv
pentahoanalysistool.csv
pfaedit.csv
dvd-create.csv
theresa.csv
ardour.csv
google-caja.csv
rocrail.csv
riff-evolve.csv
growl-for-windows.csv
ftm.csv
kftpgrabber.csv
runuomondains.csv
log4net.csv
mp-rechnungs-und-kundenverwaltung.csv
mclient-mume.csv
poormans.csv
uwom-server.csv
tycho.csv
thrust.csv
bionote.csv
emite.csv
customsagetv.csv
mediaportal.csv
lite.csv
tolven.csv
codesmith.csv
mevenide.csv
codeblocks.csv
stuproa-cims.csv
mptvseries.csv
ondex.csv
etics.csv
autofac.csv
genoviz.csv
sigil.csv
neodatis-odb.csv
unflobtactical.csv
monetdb.csv
atomsite.csv
matrex.csv
gzigzag.csv
stemkit.csv
bibedt.csv
biodwh.csv
geogebra.csv
jstock.csv
nodal.csv
                                       recall  precision    pf  wins
ardour.csv                               0.49      0.350  0.38     8
atomsite.csv                             0.55      0.370  0.34    41
autofac.csv                              0.76      0.370  0.34    48
bibedt.c

logicmail.csv
unbvision.csv
mevenide.csv
jmoney.csv
mclient-mume.csv
freedom-erp.csv
mediaportal.csv
avisynth2.csv
jam-daq.csv
pi-qmc.csv
emftriple.csv
riff-evolve.csv
fm-classic.csv
snakeyaml.csv
cascading.csv
jsecurity.csv
kftpgrabber.csv
voms.csv
gzigzag.csv
lockss.csv
carabiner.csv
re2c.csv
autat.csv
emeraldemu.csv
powerfolder-.csv
tokratan.csv
amateur-scrolls.csv
jguard.csv
qmmp.csv
maya-work-in-progress.csv
ftm.csv
metacosm.csv
net-snmp.csv
zorannt.csv
bacnet.csv
quark.csv
dataobjectsdotnet.csv
open-ms.csv
uwom.csv
biodwh.csv
olap-navigator.csv
ondex.csv
gtad.csv
ktc.csv
jstock.csv
wxcode.csv
customsagetv.csv
freewrl.csv
moving-pictures.csv
jaql.csv
modellus.csv
columba.csv
rodin-b-sharp.csv
jsr308-langtools.csv
m-a-d-n-e-s-s.csv
mptvseries.csv
gwanted.csv
rockbox.csv
co-ode-owl-plugins.csv
jrdf.csv
ambulant.csv
                           recall  precision     pf  wins
amateur-scrolls.csv          0.18      0.340  0.300    31
ambulant.csv                 0.64      0.350  0.300   

mclient-mume.csv
panda3d.csv
openefm.csv
matrex.csv
neodatis-odb.csv
wsmt.csv
sblim.csv
forester-atv.csv
atunes.csv
fluent-nhibernate.csv
amanda.csv
unflobtactical.csv
jnode.csv
playerstage.csv
fritzing.csv
multitude.csv
openmeetings.csv
jfreereport.csv
les-indemodables.csv
zealos.csv
h2database.csv
geogebra.csv
exordium.csv
taverna.csv
mediaportal.csv
ondex.csv
quark.csv
ftm.csv
ktc.csv
cornerstone.csv
sepgsql.csv
k3d.csv
open-ms.csv
mumble.csv
etics.csv
genoviz.csv
n2cms.csv
awemud.csv
acdk.csv
zoolib.csv
kftpgrabber.csv
biodwh.csv
jstock.csv
xcf.csv
powerfolder-.csv
tzod.csv
rectracker.csv
gtad.csv
zact.csv
jrdf.csv
rohxel.csv
chaosrts.csv
silvertree.csv
aubit4gl.csv
stajistics.csv
mecat.csv
brlcad.csv
alembik.csv
zildo.csv
olex2.csv
crisscross.csv
                       recall  precision    pf  wins
acdk.csv                 0.39      0.360  0.31    41
alembik.csv              0.17      0.350  0.30    37
amanda.csv               0.62      0.350  0.33    26
atunes.csv               0

jam-daq.csv
icescrum.csv
springside.csv
dsworkbench.csv
piccolo2d.csv
synecdoche.csv
atunes.csv
emftriple.csv
amanda.csv
bibedt.csv
openmeetings.csv
jfreereport.csv
voms.csv
lmms.csv
playerstage.csv
wow-qrsk.csv
phonon-vlc-mplayer.csv
mplayer-ce.csv
jaql.csv
retromenu.csv
unbvision.csv
myfaces.csv
bzbyte.csv
quark.csv
upp-mirror.csv
jajuk.csv
gss.csv
reaper-ecad.csv
sepgsql.csv
customsagetv.csv
rtb-team.csv
jcae.csv
badtrinitycore.csv
google-web-toolkit.csv
genoviz.csv
tolven.csv
kmess.csv
ktc.csv
exordium.csv
riff-evolve.csv
lportal.csv
openwbem.csv
jcl.csv
graphmak.csv
gtad.csv
optalgtoolkit.csv
tokratan.csv
ibatisdb.csv
jmol.csv
maidsafe-dht.csv
irplus.csv
codeblocks.csv
elvishrayrenderer.csv
adobe-source.csv
tzod.csv
utgb.csv
jrdf.csv
rodin-b-sharp.csv
                        recall  precision    pf  wins
adobe-source.csv          0.21       0.37  0.31    44
amanda.csv                0.50       0.31  0.33    11
atunes.csv                0.09       0.33  0.33     6
badtrinitycore.cs

jam-daq.csv
mp-rechnungs-und-kundenverwaltung.csv
springside.csv
dsworkbench.csv
sblim.csv
jgossipforum.csv
atunes.csv
fluent-nhibernate.csv
amanda.csv
jaxlib.csv
openmeetings.csv
empserver.csv
ng4j.csv
qpe.csv
fritzing.csv
voms.csv
acegisecurity.csv
jaql.csv
nodal.csv
taverna.csv
botsnscouts.csv
lportal.csv
m-a-d-n-e-s-s.csv
omseek.csv
acdk.csv
reaper-ecad.csv
pidgin.csv
customsagetv.csv
k3d.csv
applet2app.csv
adaptagrams.csv
delphilhlplib.csv
etics.csv
ossbuild.csv
tolven.csv
modellus.csv
jcae.csv
biodwh.csv
ftm.csv
mvdsv.csv
jaolt.csv
graphmak.csv
jvcl.csv
ossim.csv
co-ode-owl-plugins.csv
rohxel.csv
papertoolkit.csv
maidsafe-dht.csv
nuttx.csv
chaosrts.csv
ta3d.csv
silvertree.csv
jrdf.csv
rodin-b-sharp.csv
skia.csv
aubit4gl.csv
elvishrayrenderer.csv
pfaedit.csv
stemkit.csv
                                       recall  precision     pf  wins
acdk.csv                                0.340      0.330  0.290    32
acegisecurity.csv                       0.645      0.320  0.310    29
adap

mbse.csv
jfreereport.csv
qse.csv
turbotrader-bos.csv
pfaedit.csv
dvt.csv
jaolt.csv
modellus.csv
google-caja.csv
rocrail.csv
riff-evolve.csv
acdk.csv
ftm.csv
rosegarden.csv
runuomondains.csv
log4net.csv
mp-rechnungs-und-kundenverwaltung.csv
mclient-mume.csv
poormans.csv
uwom-server.csv
lmms.csv
dataobjectsdotnet.csv
wheat.csv
hydrate.csv
zildo.csv
mediaportal.csv
lite.csv
servicestack.csv
panda3d.csv
xcmetadataservicestoolkit.csv
bscwweasel.csv
jvcl.csv
findbugs.csv
nassp.csv
etics.csv
autofac.csv
genoviz.csv
jcae.csv
rodin-b-sharp.csv
phonon-vlc-mplayer.csv
taokgame.csv
atomsite.csv
taverna.csv
icescrum.csv
apolloplayer.csv
fritzing.csv
biodwh.csv
geogebra.csv
jstock.csv
e-bio-flow.csv
                                       recall  precision    pf  wins
acdk.csv                                 0.35      0.390  0.38    12
apolloplayer.csv                         0.43      0.380  0.33    43
atomsite.csv                             0.40      0.380  0.33    41
autofac.csv                  

personalaccess.csv
mevenide.csv
gabel.csv
triplea.csv
e-bio-flow.csv
mediaportal.csv
pustefix.csv
jam-daq.csv
pi-qmc.csv
gpsmid.csv
riff-evolve.csv
fm-classic.csv
chrysalis.csv
twostep.csv
thout.csv
kftpgrabber.csv
projecteqemu.csv
qtractor.csv
exordium.csv
mobicents.csv
re2c.csv
findbugs.csv
cornerstone.csv
powerfolder-.csv
foursquared.csv
lportal.csv
htmlunit.csv
arcallians.csv
maya-work-in-progress.csv
kmess.csv
quarkplusplus.csv
net-snmp.csv
zorannt.csv
superwaba.csv
xtreemfs.csv
img-browse-sort.csv
doomlegacy.csv
truevision.csv
genoviz.csv
olap-navigator.csv
ondex.csv
gtad.csv
ktc.csv
jstock.csv
wxcode.csv
zildo.csv
olsrd.csv
moving-pictures.csv
h2database.csv
modellus.csv
columba.csv
rodin-b-sharp.csv
jsr308-langtools.csv
m-a-d-n-e-s-s.csv
mptvseries.csv
gwanted.csv
opengs.csv
co-ode-owl-plugins.csv
jrdf.csv
ambulant.csv
                        recall  precision     pf  wins
ambulant.csv              0.57      0.440  0.360    59
arcallians.csv            0.62      0.430  0.380   

mclient-mume.csv
jam-daq.csv
jsecurity.csv
matrex.csv
neodatis-odb.csv
jmoney.csv
piccolo2d.csv
forester-atv.csv
atunes.csv
fluent-nhibernate.csv
gnome-jabber.csv
unflobtactical.csv
jnode.csv
playerstage.csv
superwaba.csv
qmmp.csv
openmeetings.csv
jfreereport.csv
acegisecurity.csv
crux-framework.csv
jaql.csv
geogebra.csv
exordium.csv
wishmaster.csv
fbmanager.csv
x-ray-mule.csv
quark.csv
ftm.csv
omseek.csv
reaper-ecad.csv
sepgsql.csv
k3d.csv
riff-evolve.csv
adaptagrams.csv
etics.csv
gabel.csv
n2cms.csv
liferea.csv
jcae.csv
zoolib.csv
kindb.csv
applet2app.csv
jstock.csv
xcf.csv
jcl.csv
moving-pictures.csv
x4x.csv
gtad.csv
zact.csv
jrdf.csv
opennms.csv
chaosrts.csv
silvertree.csv
psrchive.csv
stajistics.csv
mecat.csv
brlcad.csv
alembik.csv
zildo.csv
olex2.csv
l2j.csv
                       recall  precision     pf  wins
acegisecurity.csv        0.58      0.350  0.310    33
adaptagrams.csv          0.29      0.330  0.300    18
alembik.csv              0.09      0.330  0.280    36
applet2ap

chrysalis.csv
icescrum.csv
springside.csv
dsworkbench.csv
sblim.csv
synecdoche.csv
atunes.csv
columba.csv
amanda.csv
bibedt.csv
openmeetings.csv
jfreereport.csv
voms.csv
lmms.csv
ng4j.csv
wow-qrsk.csv
phonon-vlc-mplayer.csv
aztec.csv
jaql.csv
nodal.csv
jython.csv
fbmanager.csv
bzbyte.csv
quark.csv
bloodycore.csv
zorannt.csv
qcad2.csv
reaper-ecad.csv
sepgsql.csv
zildo.csv
rtb-team.csv
applet2app.csv
adaptagrams.csv
ftm.csv
genoviz.csv
tolven.csv
rezilla.csv
sigil.csv
mptvseries.csv
riff-evolve.csv
jstock.csv
mvdsv.csv
jcl.csv
graphmak.csv
gtad.csv
optalgtoolkit.csv
tokratan.csv
ibatisdb.csv
jmol.csv
maidsafe-dht.csv
irplus.csv
codeblocks.csv
elvishrayrenderer.csv
adobe-source.csv
theresa.csv
rockbox.csv
jrdf.csv
rodin-b-sharp.csv
                        recall  precision     pf  wins
adaptagrams.csv           0.21      0.320  0.320     8
adobe-source.csv          0.18      0.330  0.300    47
amanda.csv                0.52      0.320  0.320    26
applet2app.csv            0.72      0.320

mclient-mume.csv
jam-daq.csv
apertium.csv
springside.csv
dsworkbench.csv
jmoney.csv
freedom-erp.csv
atunes.csv
fluent-nhibernate.csv
amanda.csv
bibedt.csv
openmeetings.csv
cortex-vfx.csv
ng4j.csv
yale.csv
superwaba.csv
voms.csv
gdis.csv
jaql.csv
retromenu.csv
jmri.csv
fbmanager.csv
lportal.csv
m-a-d-n-e-s-s.csv
img-browse-sort.csv
acdk.csv
emeraldemu.csv
xstore.csv
zildo.csv
rtb-team.csv
applet2app.csv
adaptagrams.csv
gzigzag.csv
etics.csv
jparsec.csv
tolven.csv
modellus.csv
riff-evolve.csv
biodwh.csv
ftm.csv
mvdsv.csv
mia.csv
graphmak.csv
jvcl.csv
sabbu.csv
opengs.csv
opennms.csv
papertoolkit.csv
nassp.csv
nuttx.csv
chaosrts.csv
alembik.csv
silvertree.csv
jrdf.csv
rodin-b-sharp.csv
skia.csv
opensim4opencog.csv
elvishrayrenderer.csv
pfaedit.csv
stemkit.csv
                       recall  precision     pf  wins
acdk.csv                0.410      0.370  0.340    32
adaptagrams.csv         0.235      0.380  0.340    20
alembik.csv             0.290      0.380  0.330    40
amanda.csv       